In [1]:
import pandas as pd
import os
import geemap, ee
import seaborn as sns
import matplotlib.pyplot as plt
import math

In [2]:
Map = geemap.Map()
shp = ee.FeatureCollection("projects/ee-my-lsxproject/assets/China")
# f_shp = r'D:\全国基础数据\2022省级\中国_pro.shp'
# shp_ee = geemap.shp_to_ee(f_shp,encoding='utf-8')
roi = shp.geometry()
Map.addLayer(roi)
# Map

In [7]:
#读取影像
def get_data(year):
    dmsp = ee.ImageCollection("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS")\
        .filter(ee.Filter.calendarRange(year,year,'year'))
    id = dmsp.aggregate_array('system:id').getInfo()
    dmsp = ee.Image(id).select('stable_lights')
    return dmsp

# load the calibration coefficients reference file

coef = pd.read_csv(r'H:\夜间灯光\Elvidge_DMSP_intercalib_coef.csv')

# define a function that takes an image, gets its name, and returns its coefficients
def get_coefs(img, coefdata=coef):
    imgID = img.id().getInfo()
    idx = coefdata['satellite']+coefdata['year'].astype(str)==imgID
    return coefdata.loc[idx, ['c0','c1','c2']].values[0]


    
# define a function that wraps the ee.Image native "expression" function 
# and passess the image coefficients as arguments to return the calibrated image
def calibrate_img(img):
    c0, c1, c2 = get_coefs(img)
    return img.expression("c0 + (c1 * X) + (c2 * X**2)", {'X':img, 'c0':c0, 'c1':c1, 'c2':c2})

# define a function that clips images
def clip_img(img, upper_thresh=63, upper_set=63, lower_thresh=6, lower_set=0):
    return img.where(img.gt(upper_thresh),upper_set).where(img.lte(lower_thresh),lower_set)

# wrap these in a single function using defaults
def calibrate_and_clip(img):
    return clip_img(calibrate_img(img))

In [8]:
for year in range(2008,2019):
    dmsp = get_data(year)
    dmsp1996_clbr2 = calibrate_and_clip(dmsp)
    geemap.download_ee_image(dmsp1996_clbr2.clip(roi),r'H:\夜间灯光\DMSP' + os.sep + str(year) + 'DMSP.tif',region=roi,crs='EPSG:4326',scale=1000)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
a = 11.7319
b = 0.4436
def calc(img):
    img1 = img.select('avg_rad')
    img2 = img1.where(NPP.gt(63),63)
    result = img2.where(img.lt(0),0)
    result.pow(b)
    image = result * a
    return image.copyProperties(image, ["system:time_start",'system:id'])

In [10]:
for year in range(2013,2023):
    NPP = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG")\
        .filter(ee.Filter.calendarRange(year,year,'year')) \
        .select('avg_rad')\
        .mean()
#         .map(calc)
    img = NPP.where(NPP.gt(63),63)
    result = img.where(img.lt(0),0)
    geemap.download_ee_image(NPP.clip(roi),r'H:\夜间灯光\NPP' + os.sep + str(year) + 'NPP.tif',region=roi,crs='EPSG:4326',scale=1000)

2013NPP.tif: |                                                         | 0.00/152M (raw) [  0.0%] in 00:00 (et…

2014NPP.tif: |                                                         | 0.00/152M (raw) [  0.0%] in 00:00 (et…

2015NPP.tif: |                                                         | 0.00/152M (raw) [  0.0%] in 00:00 (et…

2016NPP.tif: |                                                         | 0.00/152M (raw) [  0.0%] in 00:00 (et…

2017NPP.tif: |                                                         | 0.00/152M (raw) [  0.0%] in 00:00 (et…

2018NPP.tif: |                                                         | 0.00/152M (raw) [  0.0%] in 00:00 (et…

2019NPP.tif: |                                                         | 0.00/152M (raw) [  0.0%] in 00:00 (et…

2020NPP.tif: |                                                         | 0.00/152M (raw) [  0.0%] in 00:00 (et…

2021NPP.tif: |                                                         | 0.00/152M (raw) [  0.0%] in 00:00 (et…

2022NPP.tif: |                                                         | 0.00/152M (raw) [  0.0%] in 00:00 (et…